In [ ]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
def parse_to_list(resp_json):
    """
    Parses the given JSON response containing cryptocurrency data into a list of dictionaries.
    
    Args:
        resp_json (dict): The JSON response containing cryptocurrency data.
        
    Returns:
        list: A list of dictionaries representing parsed cryptocurrency data.
    """
    # Initialize an empty list to store the parsed data
    result = []
    
    # Extract relevant information from the response JSON
    data = resp_json['data']
    id = data['id']
    name = data['name']
    symbol = data['symbol']
    
    # Iterate through each quote in the 'quotes' list
    for quote in data['quotes']:
        # Extract the 'quote' dictionary from the current quote
        quote_data = quote['quote']
        
        # Create a dictionary to hold the parsed entry
        entry = {
            'id': id,
            'name': name,
            'symbol': symbol,
            
            # Merge the 'quote_data' dictionary (quote) with the rest of the quote data
            **quote_data,
            
            # Convert timestamp-related fields to datetime objects
            'timestamp': datetime.strptime(quote_data['timestamp'], '%Y-%m-%dT%H:%M:%S.%fZ'),
            'timeOpen': datetime.strptime(quote['timeOpen'], '%Y-%m-%dT%H:%M:%S.%fZ'),
            'timeClose': datetime.strptime(quote['timeClose'], '%Y-%m-%dT%H:%M:%S.%fZ'),
            'timeHigh': datetime.strptime(quote['timeHigh'], '%Y-%m-%dT%H:%M:%S.%fZ'),
            'timeLow': datetime.strptime(quote['timeLow'], '%Y-%m-%dT%H:%M:%S.%fZ')
        }
        
        # Add the parsed entry to the result list
        result.append(entry)
    
    # Return the list of parsed data
    return result


In [ ]:
coins = {}
data_available = {}
total_list = [] 
for id in range(1, 5):
    time_start = 1262304000
    time_end = 1893456000
    interval = '30d'
    url = f"https://api.coinmarketcap.com/data-api/v3.1/cryptocurrency/historical?id={id}&convertId=2781&timeStart={time_start}&timeEnd={time_end}&interval={interval}"
    headers = {}
    
    resp = requests.get(url, headers=headers).json()
    print(id)
    if 'data' in resp:
        coin_data = resp['data']
        coin_name = coin_data.get('name')
        coin_quotes = coin_data.get('quotes')
    else:
        coin_name = None
        coin_quotes = False
        
    coins[id] = coin_name
    data_available[id] = bool(coin_quotes)
    if coin_quotes:
        total_list.extend(parse_to_list(resp))



In [ ]:
c = 0
for i, k in zip(coins.values(), data_available.values()):
    if k:
        c += 1
print(f'_____________\nTotal: {c}/{len(data_available)}\n{round(c/len(data_available), 4)*100}%')

In [ ]:
pd.DataFrame(total_list).to_csv('CoinMarketCap all coins.csv', index=False)